# Type Ia Supernova Hubble Diagram

This notebook will be used to explore the `SNe data.csv` dataset and document intermediate results for the distance conversion work.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

import SNe_Hubble as sne

In [ ]:
data = sne.load_modulus_data()
subset = data[: sne.PLOT_ENTRY_LIMIT]

if len(subset) == 0:
    raise RuntimeError("No supernova data available to process.")

modulus = subset["Distance_Modulus"]
modulus_error = subset["Distance_Modulus_Error"]
redshift = subset["Redshift"]

(distances, distance_errors) = sne.modulus_to_distance(
    modulus,
    modulus_error,
)

In [ ]:
print(f"Loaded {len(data)} supernovae using SciPy {scipy.__version__}.")
print(f"Plotting first {len(subset)} entries to {sne.PLOT_OUTPUT_PATH.name}.")
print("First entry as a quick check:")
print(
    f"{subset['Supernova_Name'][0]} | "
    f"mu = {modulus[0]:.3f} +/- {modulus_error[0]:.3f} -> "
    f"d = {distances[0]:.3e} +/- {distance_errors[0]:.3e} pc"
)

In [ ]:
sne.plot_modulus_vs_redshift(
    redshift,
    modulus,
    modulus_error,
    sne.PLOT_OUTPUT_PATH,
)
print(f"Saved plot to {sne.PLOT_OUTPUT_PATH.resolve()}")

Step Two

In [ ]:
best_fit_h0 = None
best_fit_h0_err = None
best_fit_h0_km_s_mpc = None
best_fit_h0_km_s_mpc_err = None

low_z_mask = redshift < sne.LOW_Z_THRESHOLD

filtered_low_z = None
filtered_low_d = None
filtered_low_d_err = None
low_z_rejected_points = None

if np.any(low_z_mask):
    low_z = redshift[low_z_mask]
    low_d = distances[low_z_mask]
    low_d_err = distance_errors[low_z_mask]

    low_valid_mask = (
        np.isfinite(low_z)
        & np.isfinite(low_d)
        & np.isfinite(low_d_err)
        & (low_d_err >= 0.0)
    )

    def _low_z_model(z_subset, d_subset, err_subset):
        h0, _ = sne.fit_hubble_constant(z_subset, d_subset, err_subset)
        return sne.hubble_law(z_subset, h0)

    (inlier_mask, outlier_mask) = sne._sigma_clip_outliers(
        low_z,
        low_d,
        low_d_err,
        low_valid_mask,
        _low_z_model,
        min_required_points=2,
        sigma_threshold=sne.LOW_Z_SIGMA_CLIP_THRESHOLD,
        max_iterations=sne.LOW_Z_SIGMA_CLIP_MAX_ITER,
    )

    if not np.any(low_valid_mask):
        print(
            "No finite low-z entries survived quality checks; "
            "skipping low-z fit and plot."
        )
    else:
        filtered_low_z = low_z[inlier_mask]
        filtered_low_d = low_d[inlier_mask]
        filtered_low_d_err = low_d_err[inlier_mask]
        num_outliers = int(np.count_nonzero(outlier_mask))
        low_z_rejected_points = None

        if filtered_low_z.size >= 2:
            if num_outliers:
                low_z_rejected_points = (
                    low_z[outlier_mask],
                    low_d[outlier_mask],
                    low_d_err[outlier_mask],
                )
                print(
                    f"Removed {num_outliers} low-z outlier(s) "
                    f"using {sne.LOW_Z_SIGMA_CLIP_THRESHOLD:.0f} sigma residual clipping."
                )
        else:
            filtered_low_z = low_z[low_valid_mask]
            filtered_low_d = low_d[low_valid_mask]
            filtered_low_d_err = low_d_err[low_valid_mask]
            low_z_rejected_points = None
            if num_outliers:
                print(
                    "Not enough low-z points after clipping; "
                    "using all valid measurements for the H0 fit instead."
                )
else:
    print(
        f"No entries below redshift {sne.LOW_Z_THRESHOLD}; "
        "skipping low-z plot."
    )

In [ ]:
if filtered_low_z is not None and filtered_low_z.size > 0:
    try:
        (best_fit_h0, best_fit_h0_err) = sne.fit_hubble_constant(
            filtered_low_z,
            filtered_low_d,
            filtered_low_d_err,
        )
        (
            best_fit_h0_km_s_mpc,
            best_fit_h0_km_s_mpc_err,
        ) = sne.to_km_s_per_mpc(
            best_fit_h0,
            best_fit_h0_err,
        )
        print(
            "Best-fit H0 for low-z subset: "
            f"{best_fit_h0:.3e} pc^-1 "
            f"(+/- {best_fit_h0_err:.3e}) -> "
            f"{best_fit_h0_km_s_mpc:.2f} km s^-1 Mpc^-1"
            + (
                f" +/- {best_fit_h0_km_s_mpc_err:.2f}"
                if best_fit_h0_km_s_mpc_err is not None
                else ""
            )
        )
    except Exception as exc:
        best_fit_h0 = None
        best_fit_h0_err = None
        best_fit_h0_km_s_mpc = None
        best_fit_h0_km_s_mpc_err = None
        print(f"Could not fit H0 for low-z subset: {exc}")

In [ ]:
if filtered_low_z is not None and filtered_low_z.size > 0:
    if best_fit_h0 is not None:
        sne.plot_distance_vs_redshift(
            filtered_low_z,
            filtered_low_d,
            filtered_low_d_err,
            sne.LOW_Z_PLOT_OUTPUT_PATH,
            best_fit_h0=best_fit_h0,
            best_fit_h0_error=best_fit_h0_err,
            best_fit_h0_km_s_mpc=best_fit_h0_km_s_mpc,
            best_fit_h0_km_s_mpc_error=best_fit_h0_km_s_mpc_err,
            rejected_points=low_z_rejected_points,
        )
    else:
        sne.plot_distance_vs_redshift(
            filtered_low_z,
            filtered_low_d,
            filtered_low_d_err,
            sne.LOW_Z_PLOT_OUTPUT_PATH,
        )

    print(
        f"Saved low-z (< {sne.LOW_Z_THRESHOLD}) distance plot to "
        f"{sne.LOW_Z_PLOT_OUTPUT_PATH.resolve()}"
    )

In [ ]:
h0_for_cosmo = best_fit_h0 if best_fit_h0 is not None else sne.DEFAULT_H0_PC_INV

if best_fit_h0 is None:
    print(
        "No reliable low-z H0 fit; defaulting to "
        f"{sne.DEFAULT_H0_KM_S_MPC:.1f} km s^-1 Mpc^-1 "
        "(converted to pc^-1) for cosmological optimization."
    )

In [ ]:
cosmo_valid_mask = (
    np.isfinite(redshift)
    & np.isfinite(distances)
    & np.isfinite(distance_errors)
    & (redshift >= 0.0)
    & (distances > 0.0)
    & (distance_errors >= 0.0)
)

filtered_redshift = None
filtered_distances = None
filtered_distance_errors = None
filtered_modulus = None
filtered_modulus_error = None
cosmo_rejected_points = None

if not np.any(cosmo_valid_mask):
    raise RuntimeError("No valid entries available for cosmological fit.")


def _cosmo_model(z_subset, d_subset, err_subset):
    (omega_m, omega_lambda) = sne.fit_density_parameters(
        z_subset,
        d_subset,
        err_subset,
        h0_for_cosmo,
    )
    return sne.predict_luminosity_distance(
        z_subset,
        omega_m,
        omega_lambda,
        h0_for_cosmo,
    )


try:
    (cosmo_inlier_mask, cosmo_outlier_mask) = sne._sigma_clip_outliers(
        redshift,
        distances,
        distance_errors,
        cosmo_valid_mask,
        _cosmo_model,
        min_required_points=3,
        sigma_threshold=sne.COSMO_SIGMA_CLIP_THRESHOLD,
        max_iterations=sne.COSMO_SIGMA_CLIP_MAX_ITER,
    )
except Exception as exc:
    print(f"Could not run cosmological sigma clipping: {exc}")
    cosmo_inlier_mask = cosmo_valid_mask.copy()
    cosmo_outlier_mask = np.zeros_like(redshift, dtype=bool)

cosmo_filtered_mask = cosmo_inlier_mask.copy()
num_cosmo_inliers = int(np.count_nonzero(cosmo_filtered_mask))
num_cosmo_outliers = int(np.count_nonzero(cosmo_outlier_mask))
use_all_cosmo_data = False

if num_cosmo_inliers < 3:
    cosmo_filtered_mask = cosmo_valid_mask
    use_all_cosmo_data = True
    if num_cosmo_outliers:
        print(
            "Not enough cosmological points after clipping; "
            "using all valid entries for the Ω fit instead."
        )
    num_cosmo_outliers = 0

if num_cosmo_outliers and not use_all_cosmo_data:
    cosmo_rejected_points = (
        redshift[cosmo_outlier_mask],
        modulus[cosmo_outlier_mask],
        modulus_error[cosmo_outlier_mask],
    )
    print(
        f"Removed {num_cosmo_outliers} cosmological outlier(s) "
        f"using {sne.COSMO_SIGMA_CLIP_THRESHOLD:.0f} sigma residual clipping."
    )

filtered_redshift = redshift[cosmo_filtered_mask]
filtered_distances = distances[cosmo_filtered_mask]
filtered_distance_errors = distance_errors[cosmo_filtered_mask]
filtered_modulus = modulus[cosmo_filtered_mask]
filtered_modulus_error = modulus_error[cosmo_filtered_mask]

In [ ]:
try:
    (omega_m_fit, omega_lambda_fit) = sne.fit_density_parameters(
        filtered_redshift,
        filtered_distances,
        filtered_distance_errors,
        h0_for_cosmo,
    )
    print(
        "Best-fit density parameters from cosmological d_L(z): "
        f"Omega_M = {omega_m_fit:.3f}, Omega_Lambda = {omega_lambda_fit:.3f}"
    )
    sne.plot_cosmological_modulus_fit(
        filtered_redshift,
        filtered_modulus,
        filtered_modulus_error,
        omega_m_fit,
        omega_lambda_fit,
        h0_for_cosmo,
        sne.COSMO_MODULUS_PLOT_OUTPUT_PATH,
        rejected_points=cosmo_rejected_points,
    )
    print(
        "Saved cosmological distance-modulus plot to "
        f"{sne.COSMO_MODULUS_PLOT_OUTPUT_PATH.resolve()}"
    )
except Exception as exc:
    print(f"Could not perform cosmological fit: {exc}")